In [ ]:
#!/usr/bin/env python
# coding: utf-8

In[196]:

In [ ]:
from elasticsearch import Elasticsearch, helpers
from ssl import create_default_context
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import parametros
import random
import re
import time

In [ ]:
# ### ¿Qué hace este script?
# 
# #### Para cada AP:
# * usuarios.usuariosConectados(corresponde al total de conexiones), usuarios.sesiones_Usuarios (Corresponde al total de usuarios)
# * Información detallada de los dispositivos conectados en el rango de fecha procesado: usuarios.dispositivo.mac, usuarios.dispositivo.tipo, usuarios.dispositivo.marca, usuarios.dispositivo.tecnologia, usuarios.dispositivo.sisOperativo.
# 
# #### Para cada sitio: 
# * Totales por características de dispositivos: usuarios.sistemaOperativoUsuarios, usuarios.tipoDispositivoUsuarios, usuarios.marcaTerminal, usuarios.tecnologiaTerminal, usuarios.totales.dispositivos (Este ultimo guarda el total)
# * usuarios.usoServicioInternetSitio es la suma en Gb consumido discriminado por usuarios.detallesTecnologiasTerminales (Bandas 2.4 o 5 GHz)
# * usuarios.conteoDispositivos el cual indica el total de dispositivos conectados (Es lo mismo que total de conexiones)
# * usuarios.usuariosNuevos (Esta tenía antes Nuevos y Recurrentes, pero ahora solo tiene Nuevos por solicitud de BI)
# * usuarios.totales.usuariosNuevos, tiene el total de usuarios nuevos (el script por el momento solo totaliza nuevos siempre y cuando no se corra historicos)

## Conectando a ElasticSearch

La ultima lÃ­nea se utiliza para garantizar la ejecuciÃ³n de la consulta<br>
* timeout es el tiempo para cada ejecuciÃ³n<br>
* max_retries el nÃºmero de intentos si la conexiÃ³n falla<br>
* retry_on_timeout para activar los reitentos

In[197]:

In [ ]:
context = create_default_context(cafile=parametros.cafile)
es = Elasticsearch(
    parametros.servidor,
    http_auth=(parametros.usuario_EC, parametros.password_EC),
    scheme="https",
    port=parametros.puerto,
    ssl_context=context,
    timeout=60, max_retries=3, retry_on_timeout=True
)

### Calculando fechas para la ejecuciÃ³n

* Se calculan las fechas para asociar al nombre del indice<br>
* fecha_hoy es usada para concatenar al nombre del indice principal previa inserciÃ³n

In[198]:

In [ ]:
now = datetime.now()
fecha_hoy = str(now.strftime("%Y.%m.%d"))

### nombre de indice donde se insertarÃ¡

In[199]:

In [ ]:
indice = parametros.usuarios_tablero09_index 
indice_control = parametros.tableros_mintic_control

### Funcion para construir JSON compatible con ElasticSearch

In[200]:

In [ ]:
def filterKeys(document, use_these_keys):
    return {key: document.get(key) for key in use_these_keys }

### Trae la ultima fecha para control de ejecuciÃ³n

Cuando en el rango de tiempo de la ejecuciÃ³n, no se insertan nuevos valores, las fecha maxima en indice mintic no aumenta, por tanto se usa esta fecha de control para garantizar que incremente el bucle de ejecuciÃ³n

In[201]:

In [ ]:
total_docs = 1
try:
    response = es.search(
        index= indice_control,
        body={
           "_source": ["tablero09.fechaControl"],
              "query": {
                "bool": {
                  "filter": [
                  {
                    "exists": {
                      "field":"jerarquia-tablero09"
                    }
                  }
              ]
            }
          }
        },
        size=total_docs
    )
    #print(es.info())
    elastic_docs = response["hits"]["hits"]
    fields = {}
    for num, doc in enumerate(elastic_docs):
        fecha_ejecucion = doc["_source"]['tablero09.fechaControl']
except Exception as e:
    print(e)
    response["hits"]["hits"] = []
if response["hits"]["hits"] == []:
    fecha_ejecucion = '2021-05-14 00:00:00'
print("ultima fecha para control de ejecucion:",fecha_ejecucion)

### leyendo indice semilla-inventario

En el script que ingesta semilla, trae la informaciÃ³n de los centros de conexiÃ³n administrados. Para el indice principal se requiere:<br>
* site_id como llave del centro de conexiÃ³n.<br>
* Datos geogrÃ¡ficos (Departamento, municipio, centro poblado, sede, energÃ­a, latitud, longitud,COD_ISO, id_Beneficiario).

In[202]:

In [ ]:
t1=time.time()
total_docs = 10000
try:
    response = es.search(
        index= parametros.semilla_inventario_index,
        body={
               "_source": ['site_id','nombre_municipio', 'nombre_departamento', 'nombre_centro_pob', 'nombreSede' 
                           , 'energiadesc', 'latitud', 'longitud', 'COD_ISO','id_Beneficiario']
        },
        size=total_docs
    )
    #print(es.info())
    elastic_docs = response["hits"]["hits"]
#     fields = {}
#     for num, doc in enumerate(elastic_docs):
#         source_data = doc["_source"]
#         for key, val in source_data.items():
#             try:
#                 fields[key] = np.append(fields[key], val)
#             except KeyError:
#                 fields[key] = np.array([val])

#     datos_semilla = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in fields.items() ])) #pd.DataFrame(fields)
    
    datos_semilla = pd.DataFrame([x["_source"] for x in elastic_docs])
    
except:
    exit()
    
t2=time.time()

Se eliminan espacios en blanco

In[203]:

In [ ]:
datos_semilla['site_id'] = datos_semilla['site_id'].str.strip()

### Cambiando nombre de campos y generando location

* Se valida latitud y longitud. Luego se calcula campo location<br>
* Se renombran los campos de semilla

In[204]:

In [ ]:
def get_location(x,y='lat'):
    patron = re.compile('^(\-?\d+(\.\d+)?),\s*(\-?\d+(\.\d+)?)$') #patrÃ³n que debe cumplir
    if (not patron.match(x) is None) and (str(x)!=''):
        return x.replace(',','.')
    else:
        #CÃ³digo a ejecutar si las coordenadas no son vÃ¡lidas
        return '4.596389' if y=='lat' else '-74.074639'
    
datos_semilla['latitud'] = datos_semilla['latitud'].apply(lambda x:get_location(x,'lat'))
datos_semilla['longitud'] = datos_semilla['longitud'].apply(lambda x:get_location(x,'lon'))
# datos_semilla['latitud'] = datos_semilla['latitud'].apply(get_location)
# datos_semilla['longitud'] = datos_semilla['longitud'].apply(get_location)
datos_semilla = datos_semilla.drop(datos_semilla[(datos_semilla["longitud"]=='a') | (datos_semilla["latitud"]=='a')].index)
datos_semilla['usuarios.location'] = datos_semilla['latitud'] + ',' + datos_semilla['longitud']
datos_semilla['usuarios.location']=datos_semilla['usuarios.location'].str.replace('a,a','')
datos_semilla.drop(columns=['latitud','longitud'],inplace=True)

In[205]:

In [ ]:
datos_semilla = datos_semilla.rename(columns={'lugar_cod' : 'usuarios.centroDigitalUsuarios'
                                            , 'nombre_municipio': 'usuarios.nombreMunicipio'
                                            , 'nombre_departamento' : 'usuarios.nombreDepartamento'
                                            , 'nombre_centro_pob': 'usuarios.localidad'
                                            , 'nombreSede' : 'usuarios.nomCentroDigital'
                                            , 'energiadesc' : 'usuarios.sistemaEnergia'
                                            , 'COD_ISO' : 'usuarios.codISO'
                                            , 'id_Beneficiario' : 'usuarios.idBeneficiario'})

Se descartan los registros que tengan la latitud y longitud vacÃ­a o no valida

In[206]:

In [ ]:
datos_semilla = datos_semilla.drop(datos_semilla[(datos_semilla["usuarios.location"]=='')].index)

### leyendo indice cambium-devicedevices

De esta formas se asocia las MAC de dispositivos de red INDOOR y OUTDOOR<br>
* site_id para cruzar con las misma llave de semilla.<br>
* datos del dispositivo: mac, status, ip.<br>
* ap_group para identificar si la conexiÃ³n es indoor/outdoor

In[207]:

In [ ]:
t1=time.time()
total_docs = 30000
try:
    response = es.search(
        index= parametros.cambium_d_d_index,
        body={
                    "_source": ["site_id","mac","status","ip","ap_group"]
                  , "query": {
                    "match_all": {}
                  }
        },
        size=total_docs
    )
    #print(es.info())
    elastic_docs = response["hits"]["hits"]
#     fields = {}
#     for num, doc in enumerate(elastic_docs):
#         source_data = doc["_source"]
#         for key, val in source_data.items():
#             try:
#                 fields[key] = np.append(fields[key], val)
#             except KeyError:
#                 fields[key] = np.array([val])

#     datos_dev = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in fields.items() ])) #pd.DataFrame(fields)
    
    datos_dev = pd.DataFrame([x["_source"] for x in elastic_docs])
    
    
except:
    exit()
    
t2=time.time()

In[208]:

In [ ]:
datos_dev['site_id'] = datos_dev['site_id'].str.strip()

Se descartan registros con site_id vacios ya que no cruzarÃ¡n en el merge y se limpian los NaN del dataframe.

In[209]:

In [ ]:
datos_dev.dropna(subset=['site_id'], inplace=True)
datos_dev.fillna('', inplace=True)
datos_dev = datos_dev.drop(datos_dev[(datos_dev['site_id']=='')].index)

In[210]:

In [ ]:
datos_dev['ap_group'] = datos_dev['ap_group'].str.split("-", n = 1, expand = True)[0]
datos_dev['ap_group'] = datos_dev['ap_group'].str.split("_", n = 1, expand = True)[0]
datos_dev['ap_group'] = datos_dev['ap_group'].str.split(".", n = 1, expand = True)[0]
datos_dev = datos_dev.drop(datos_dev[(datos_dev['ap_group']=='')].index)

Se toman solo los datos unicos con mac.

In[211]:

atos_dev.drop_duplicates(subset=['site_id','mac'],inplace=True)

In [ ]:
datos_dev = datos_dev.drop_duplicates('mac')

Se cambia el nombre a la mac del dispositivo de red para no confundir con la de dispositivos de usuario 

In[212]:

In [ ]:
datos_dev= datos_dev.rename(columns={'mac' : 'usuarios.macRed','ap_group' : 'usuarios.apGroup'})

## Lectura de datos ohmyfi-detalleconexiones

Los datos que se toman son:<br>
* fechahora (de cada conexiÃ³n). fecha_control es lo mismo pero con el intervalo de 10 minutos<br>
* InformaciÃ³n del centro: lugar, lugar_cod.<br>
* InformaciÃ³n del dispositivo de usuario: mac_usuario, dispositivo, sistema_operativo.<br>
* InformaciÃ³n de usuario: tipodoc y documento

In[213]:

In [ ]:
def trae_conexiones(fecha_ini,fecha_fin):
    total_docs = 5000000
    response = es.search(
        index= parametros.ohmyfi_d_c_index,
        body={
                "_source": ["fechahora","fecha_control","lugar","lugar_cod","mac_usuario", "dispositivo"
                            ,"sistema_operativo",'tipodoc','documento']
                , "query": {
                  "range": {
                    "fechahora": {
                      "gte": fecha_ini,
                      "lt": fecha_fin
                    }
                  }
              }
        },
        size=total_docs
    )
    elastic_docs = response["hits"]["hits"]
#     fields = {}
#     for num, doc in enumerate(elastic_docs):
#         source_data = doc["_source"]
#         for key, val in source_data.items():
#             try:
#                 fields[key] = np.append(fields[key], val)
#             except KeyError:
#                 fields[key] = np.array([val])

#     return pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in fields.items() ])) 

    return pd.DataFrame([x["_source"] for x in elastic_docs])

### Lanzando ejecuciÃ³n de consulta

* Se calcula rango en base a la fecha de control. Para este caso es de 10 minutos.<br>
* Se ejecuta la funciÃ³n de consulta con el rango de fechas.<br>
* Si no retorna datos se incrementa el rango y se ejecuta nuevamente. Este proceso se repite hasta conseguir datos o hasta que el rango de ejecuciÃ³n alcance la fecha y hora actual.<br>


In[214]:

In [ ]:
fecha_max_mintic = fecha_ejecucion
fecha_tope_mintic = (datetime.strptime(fecha_max_mintic, "%Y-%m-%d %H:%M:%S")+timedelta(minutes=50)-timedelta(seconds=1)).strftime("%Y-%m-%d %H:%M:%S")
datos_det_conex = trae_conexiones(fecha_max_mintic,fecha_tope_mintic)

In [ ]:
if datos_det_conex is None or datos_det_conex.empty:
    while (datos_det_conex is None or datos_det_conex.empty) and ((datetime.strptime(fecha_max_mintic[0:10], '%Y-%m-%d').strftime("%Y-%m-%d %H:%M:%S")) < str(now.strftime("%Y-%m-%d %H:%M:%S"))):
        fecha_max_mintic = (datetime.strptime(fecha_max_mintic, "%Y-%m-%d %H:%M:%S")+timedelta(minutes=50)).strftime("%Y-%m-%d %H:%M:%S")
        fecha_tope_mintic = (datetime.strptime(fecha_tope_mintic, "%Y-%m-%d %H:%M:%S")+timedelta(minutes=50)).strftime("%Y-%m-%d %H:%M:%S")
        datos_det_conex = trae_conexiones(fecha_max_mintic,fecha_tope_mintic)
else:
    pass
                                                                                                             

In[215]:

In [ ]:
datos_det_conex['fecha'] = datos_det_conex['fecha_control'].str.split(" ", n = 1, expand = True)[0]
datos_det_conex.drop_duplicates(subset=["fecha_control","lugar","lugar_cod","mac_usuario", "dispositivo","sistema_operativo",'tipodoc','documento'],inplace=True)

In[216]:

In [ ]:
datos_det_conex = datos_det_conex.rename(columns={'lugar_cod' : 'site_id'
                                                   ,'fechahora':'usuarios.fechaConexionUsuarios'
                                                   ,'dispositivo': 'usuarios.tipoDispositivoUsuarios'
                                                   , 'sistema_operativo': 'usuarios.sistemaOperativoUsuarios'})

Se corrigen valores errados de site_id en detalle conexiones

### Se lee informaciÃ³n de Ohmyfi consumos

Apartir de esta lectura se toma el valor del tiempo promedio de sesiÃ³n en minutos. Se toma de referencia los campos:<br>
* fecha_inicio a partir de la cual se calcula fecha control<br>
* tiempo_sesion_minutos<br>
* mac_ap<br>
* mac_usuario <br>
* lugar_cod

In[217]:

In [ ]:
def traeSesiones(fecha_max,fecha_tope):
    total_docs = 500000
    response = es.search(
        index= parametros.ohmyfi_consumos_index,
        body={
                  "_source": ["lugar_cod", "tiempo_sesion_minutos","mac_ap","mac_usuario","fecha_inicio"]
                , "query": {
                      "range": {
                            "fecha_inicio": {
                            "gte": fecha_max,
                            "lt": fecha_tope
                            }
                        }
                  }
        },
        size=total_docs
    )
    elastic_docs = response["hits"]["hits"]
#     fields = {}
#     for num, doc in enumerate(elastic_docs):
#         source_data = doc["_source"]
#         for key, val in source_data.items():
#             try:
#                 fields[key] = np.append(fields[key], val)
#             except KeyError:
#                 fields[key] = np.array([val])

#     return pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in fields.items() ])) 

    return pd.DataFrame([x["_source"] for x in elastic_docs])

### Se lee la informaciÃ³n de cambium device client

Esta lectura se usa para identificar: <br>
* Lecturas de consumo por dispositivo (radios rx y tx)<br>
* Fabricante del dispositivo<br>
* la mac del ap que luego se usa para identificar el ap group(Indoor/Outdoor)

In[218]:

In [ ]:
t1=time.time()

total_docs = 30000000
response = es.search(
    index= parametros.cambium_d_c_index,
    body={
            "_source": ["mac","ap_mac", 'manufacturer','name',"radio.band",'radio.rx_bytes','radio.tx_bytes'],
              "query": {
                "bool": {
                  "filter": [
                  {
                    "exists": {
                      "field":"mac"
                    }
                  }
                  ]
                }
              }
    },
    size=total_docs
)
elastic_docs = response["hits"]["hits"]
# fields = {}
# for num, doc in enumerate(elastic_docs):
#     source_data = doc["_source"]
#     for key, val in source_data.items():
#         try:
#             fields[key] = np.append(fields[key], val)
#         except KeyError:
#             fields[key] = np.array([val])

# datos_dev_clients = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in fields.items() ]))

datos_dev_clients = pd.DataFrame([x["_source"] for x in elastic_docs])

t2=time.time()

* se cruza por mac_usuario<br>
* Los merge usan left para evitar perdida de datos en cruce con cambium-devicedevices. Aquellos datos que no cruzan se les marca como no identificados. En condiciones ideales, no deberÃ­a presentarse ausencia de informaciÃ³n<br>
* Solo se toma lo que cruza con INNER<br>
* se cambia el fabricante [Local MAC]<br>
* se cambian lo nulos

In[219]:

In [ ]:
try:
    datos_dev_clients.drop_duplicates(inplace=True)
    datos_dev_clients = datos_dev_clients.rename(columns={'ap_mac' : 'usuarios.macRed','mac' : 'mac_usuario'})
    datos_dev_clients['manufacturer'] = datos_dev_clients['manufacturer'].replace("[Local MAC]", "No identificado")
    datos_dev_clients.fillna({'manufacturer':'No identificado'},inplace=True)
    
    datos_det_conex = pd.merge(datos_det_conex,  datos_dev_clients, on='mac_usuario',how='left')
    datos_det_conex = pd.merge(datos_det_conex,datos_dev[['usuarios.macRed','usuarios.apGroup']],on='usuarios.macRed', how='left')
    
    datos_det_conex.fillna({'usuarios.tipoDispositivoUsuarios':'No identificado'
                       ,'usuarios.sistemaOperativoUsuarios':'No identificado'
                       ,'manufacturer':'No identificado'
                       ,'radio.band':'No identificado'
                       },inplace=True)
except:
    pass

## Calculando totales por dispositivos

la lista use_these_keys se usa para referenciar cuales campos del dataframe irÃ¡n al indice final. si los datos no se declaran en este, no se insertarÃ¡n

In[220]:

use_these_keys = [ 'usuarios.tablero11.siteID'<br>
                  , 'usuarios.tablero11.nomCentroDigital'<br>
                  , 'usuarios.tablero11.nombreDepartamento'<br>
                  , 'usuarios.tablero11.nombreMunicipio'<br>
                  , 'usuarios.tablero11.idBeneficiario'<br>
                  , 'usuarios.tablero11.location'<br>
                  , 'usuarios.tablero11.fechaControl'<br>
                  , 'usuarios.tablero11.fecha'<br>
                  , 'usuarios.tablero11.anyo'<br>
                  , 'usuarios.tablero11.mes'<br>
                  , '@timestamp']

def doc_generator(df,use_these_keys):<br>
    df_iter = df.iterrows()<br>
    for index, document in df_iter:<br>
        yield {<br>
                "_index": indice, <br>
                "_id": f"{ 'totalesDispositivos-' + str(document['usuarios.tablero11siteID']) + '-' + str(document['usuarios.fechaControl'])}",<br>
                "_source": filterKeys(document,use_these_keys),<br>
            }

Se agrupa por:<br>
* Sistema operativo<br>
* Tipo dispositivo<br>
* Marca del terminal<br>
* TecnologÃ­a terminal<br>
<br>
Se cuentan las mac de usuario para cada site id y fecha control

In[221]:

def cambioVariable(df):<br>
    df = df.rename(columns={'siteID':'usuarios.tablero09.siteID'<br>
                        ,'usuarios.siteID': 'usuarios.tablero09.siteID'<br>
                        ,'usuarios.tablero09siteID':'usuarios.tablero09.siteID'<br>
                        ,'fechaControl':'usuarios.tablero09.fechaControl'<br>
                        ,'usuarios.fechaControl':'usuarios.tablero09.fechaControl'<br>
                        ,'usuarios.tablero09fechaControl': 'usuarios.tablero09.fechaControl'<br>
                        ,'nomCentroDigital':'usuarios.tablero09.nomCentroDigital'<br>
                        ,'usuarios.nomCentroDigital':'usuarios.tablero09.nomCentroDigital'<br>
                        ,'usuarios.tablero09nomCentroDigital':'usuarios.tablero09.nomCentroDigital'<br>
                        ,'localidad':'usuarios.tablero09.localidad'<br>
                        ,'usuarios.localidad':'usuarios.tablero09.localidad'<br>
                        ,'usuarios.tablero09localidad':'usuarios.tablero09.localidad'<br>
                        ,'nombreDepartamento':'usuarios.tablero09.nombreDepartamento'<br>
                        ,'usuarios.nombreDepartamento':'usuarios.tablero09.nombreDepartamento'<br>
                        ,'usuarios.tablero09nombreDepartamento':'usuarios.tablero09.nombreDepartamento'<br>
                        ,'nombreMunicipio':'usuarios.tablero09.nombreMunicipio'<br>
                        ,'usuarios.nombreMunicipio':'usuarios.tablero09.nombreMunicipio'<br>
                        ,'usuarios.tablero09nombreMunicipio':'usuarios.tablero09.nombreMunicipio'<br>
                        ,'idBeneficiario':'usuarios.tablero09.idBeneficiario'<br>
                        ,'usuarios.idBeneficiario':'usuarios.tablero09.idBeneficiario'<br>
                        ,'usuarios.tablero09idBeneficiario':'usuarios.tablero09.idBeneficiario'<br>
                        ,'apGroup':'usuarios.tablero09.apGroup'<br>
                        ,'usuarios.apGroup':'usuarios.tablero09.apGroup'<br>
                        ,'usuarios.tablero09apGroup':'usuarios.tablero09.apGroup'<br>
                        ,'location':'usuarios.tablero09.location'<br>
                        ,'usuarios.location':'usuarios.tablero09.location'<br>
                        ,'usuarios.tablero09location':'usuarios.tablero09.location'<br>
                        ,'fecha':'usuarios.tablero09.fecha'<br>
                        ,'usuarios.fecha':'usuarios.tablero09.fecha'<br>
                        ,'usuarios.tablero09fecha':'usuarios.tablero09.fecha'<br>
                        ,'anyo':'usuarios.tablero09.anyo'<br>
                        ,'usuarios.anyo':'usuarios.tablero09.anyo'<br>
                        ,'usuarios.tablero09anyo':'usuarios.tablero09.anyo'<br>
                        ,'mes':'usuarios.tablero09.mes'<br>
                        ,'usuarios.mes':'usuarios.tablero09.mes'<br>
                        ,'usuarios.tablero09mes':'usuarios.tablero09.mes'<br>
                        ,'macRed':'usuarios.tablero09.macRed'<br>
                        ,'usuarios.macRed':'usuarios.tablero09.macRed'<br>
                        ,'usuarios.tablero09macRed':'usuarios.tablero09.macRed'<br>
                        ,'usuariosConectados':'usuarios.tablero09.usuariosConectados'<br>
                        ,'usuarios.usuariosConectados':'usuarios.tablero09.usuariosConectados'<br>
                        ,'usuarios.tablero09usuariosConectados':'usuarios.tablero09.usuariosConectados'<br>
                        ,'sesiones_Usuarios':'usuarios.tablero09.sesiones_Usuarios'<br>
                        ,'usuarios.sesiones_Usuarios':'usuarios.tablero09.sesiones_Usuarios'<br>
                        ,'usuarios.tablero09sesiones_Usuarios':'usuarios.tablero09.sesiones_Usuarios'<br>
                        ,'consumoUsuarios':'usuarios.tablero09.consumoUsuarios'<br>
                        ,'usuarios.consumoUsuarios':'usuarios.tablero09.consumoUsuarios'<br>
                        ,'usuarios.tablero09consumoUsuarios':'usuarios.tablero09.consumoUsuarios'<br>
                        ,'consumoUsuariosDescarga_aux':'usuarios.tablero09.consumoUsuariosDescarga_aux'<br>
                        ,'usuarios.consumoUsuariosDescarga_aux':'usuarios.tablero09.consumoUsuariosDescarga_aux'<br>
                        ,'usuarios.tablero09consumoUsuariosDescarga_aux':'usuarios.tablero09.consumoUsuariosDescarga_aux'<br>
                        ,'consumoUsuariosCarga_aux':'usuarios.tablero09.consumoUsuariosCarga_aux'<br>
                        ,'usuarios.consumoUsuariosCarga_aux':'usuarios.tablero09.consumoUsuariosCarga_aux'<br>
                        ,'usuarios.tablero09consumoUsuariosCarga_aux':'usuarios.tablero09.consumoUsuariosCarga_aux'<br>
                        ,'consumoUsuariosDescarga':'usuarios.tablero09.consumoUsuariosDescarga'<br>
                        ,'usuarios.consumoUsuariosDescarga':'usuarios.tablero09.consumoUsuariosDescarga'<br>
                        ,'usuarios.tablero09consumoUsuariosDescarga':'usuarios.tablero09.consumoUsuariosDescarga'<br>
                        ,'consumoUsuariosCarga': 'usuarios.tablero09.consumoUsuariosCarga'<br>
                        ,'usuarios.tablero09consumoUsuariosCarga': 'usuarios.tablero09.consumoUsuariosCarga'<br>
                        ,'usuarios.tiempoPromedioSesionSitio':'usuarios.tablero09.tiempoPromedioSesionSitio'<br>
                       }<br>
              )

In [ ]:
    
#     df_vacio = pd.DataFrame(index=df.index)
    
#     col_select= ['usuarios.tablero09.siteID'
#                  ,'usuarios.tablero09.nomCentroDigital'
#                  , 'usuarios.tablero09.localidad'
#                  , 'usuarios.tablero09.nombreDepartamento'
#                  , 'usuarios.tablero09.nombreMunicipio'
#                  , 'usuarios.tablero09.idBeneficiario'
#                  , 'usuarios.tablero09.location'
#                  , 'usuarios.tablero09.sesiones_Usuarios'
#                  , 'usuarios.tablero09.usuariosConectados'
#                  , 'usuarios.tablero09.consumoUsuarios'
#                  , 'usuarios.tablero09.consumoUsuariosDescarga_aux'
#                  , 'usuarios.tablero09.consumoUsuariosCarga_aux'
#                  , 'usuarios.tablero09.consumoUsuariosDescarga'
#                  , 'usuarios.tablero09.consumoUsuariosCarga'
#                  , 'usuarios.tablero09.tiempoPromedioSesionSitio'
#                  , 'usuarios.tablero09.usoServicioInternetSitio'
#                  , 'usuarios.tablero09.anchoBandaDescarga'
#                  , 'usuarios.tablero09.anchoBandaCarga'
#                  , 'usuarios.tablero09.apGroup'
#                  , 'usuarios.tablero09.fechaControl'
#                  , 'usuarios.tablero09.fecha'
#                  , 'usuarios.tablero09.anyo'
#                  , 'usuarios.tablero09.mes'
#                  , 'usuarios.tablero09.macRed'
#                  , '@timestamp']
    
#     for c in col_select:
#         try:
#             df_vacio=pd.concat([df_vacio,df[c]],axis=1)
#         except:
#             pass
    
# #     print(df.columns)
    
#     df_vacio=df_vacio.iloc[:,~df_vacio.columns.duplicated()]

    return df_vacio

In[222]:

In [ ]:
try:
    totalesDispositivos = datos_det_conex[["fecha_control","site_id"
                                         ,"usuarios.sistemaOperativoUsuarios"
                                         ,'usuarios.tipoDispositivoUsuarios'
                                         ,'manufacturer'
                                         ,'radio.band'
                                         ,"mac_usuario"]].groupby(["fecha_control","site_id"
                                                             ,"usuarios.sistemaOperativoUsuarios"
                                                             ,'usuarios.tipoDispositivoUsuarios'
                                                             ,'manufacturer'
                                                             ,'radio.band']).agg(['count']).reset_index()
    ##si se quiere contar ocurrencias unicas se debe usar en lugar del agg: ['mac_usuario'].nunique().reset_index()
    totalesDispositivos.columns = totalesDispositivos.columns.droplevel(1)
    totalesDispositivos= totalesDispositivos.rename(columns={'mac_usuario' : 'usuarios.totales.dispositivos'
                                                            ,'fecha_control' : 'usuarios.fechaControl'
                                                            ,'manufacturer' : 'usuarios.marcaTerminal'
                                                            ,'radio.band' : 'usuarios.tecnologiaTerminal'
                                                            })
    totalesDispositivos = pd.merge(totalesDispositivos, datos_semilla, on='site_id',how='inner')
    totalesDispositivos["usuarios.fecha"] = totalesDispositivos["usuarios.fechaControl"].str.split(" ", n = 1, expand = True)[0]
    totalesDispositivos["usuarios.anyo"] = totalesDispositivos["usuarios.fecha"].str[0:4]
    totalesDispositivos["usuarios.mes"] = totalesDispositivos["usuarios.fecha"].str[5:7]
    totalesDispositivos["usuarios.dia"] = totalesDispositivos["usuarios.fecha"].str[8:10]
    totalesDispositivos["usuarios.hora"] = totalesDispositivos["usuarios.fechaControl"].str.split(" ", n = 1, expand = True)[1].str.split(":", n = 2, expand = True)[0]
    totalesDispositivos["usuarios.minuto"] = totalesDispositivos["usuarios.fechaControl"].str.split(" ", n = 1, expand = True)[1].str.split(":", n = 2, expand = True)[1]
    totalesDispositivos= totalesDispositivos.rename(columns={'site_id' : 'usuarios.siteID'})
    totalesDispositivos['nombreDepartamento'] = totalesDispositivos['usuarios.nombreDepartamento']
    totalesDispositivos['nombreMunicipio'] = totalesDispositivos['usuarios.nombreMunicipio']
    totalesDispositivos['idBeneficiario'] = totalesDispositivos['usuarios.idBeneficiario']
    totalesDispositivos['fecha'] = totalesDispositivos['usuarios.fecha']
    totalesDispositivos['anyo'] = totalesDispositivos['usuarios.anyo']
    totalesDispositivos['mes'] = totalesDispositivos['usuarios.mes']
    totalesDispositivos['dia'] = totalesDispositivos['usuarios.dia']
    totalesDispositivos['@timestamp'] = now.isoformat()
    
    #Se comenta escritura en el indice porque no se necesita esa estructura
    
    totalesDispositivos = cambioVariable(totalesDispositivos)
    
   

    #salida = helpers.bulk(es, doc_generator(totalesDispositivos))
#     print("Fecha: ", now,"- Totales por Dispositivos insertados en indice principal:",salida[0])
except:
    pass
#     print("Fecha: ", now,"- No se insertaron totales por dispositivos en indice principal")
    

# Insertando usuarios conectados al indice principal

* Se calcula cantidad de sesiones por sitio con detalle conexiones, pero contando la ocurrencia unica de documento<br>
* Se calcula la cantidad de conexiones. Se agrupa por el campo usuarios.macRed(el cual corresponde al AP) y fecha_control. Se cuenta las ocurrencias de mac_usuario. Luego al cruzar con flujo principal, si el dato es nulo para ese momento, se debe colocar en 0.

In[223]:

In [ ]:
fecha_max_mintic = fecha_ejecucion

fecha_tope_mintic = (datetime.strptime(fecha_max_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(minutes=50)-timedelta(seconds=1)).strftime("%Y-%m-%d %H:%M:%S")
datos_consumos = traeSesiones(fecha_max_mintic,fecha_tope_mintic)

try:

    if datos_consumos is None or datos_consumos.empty:
        while (datos_consumos is None or datos_consumos.empty) and ((datetime.strptime(fecha_max_mintic[0:50], '%Y-%m-%d %H:%M:%S"').strftime("%Y-%m-%d %H:%M:%S")) < str(now.strftime("%Y-%m-%d %H:%M:%S"))):
            fecha_max_mintic = (datetime.strptime(fecha_max_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(minutes=50)).strftime("%Y-%m-%d %H:%M:%S")
            fecha_tope_mintic = (datetime.strptime(fecha_tope_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(minutes=50)).strftime("%Y-%m-%d %H:%M:%S")
            datos_consumos = traeSesiones(fecha_max_mintic,fecha_tope_mintic)
    else:
        pass
except:
    pass

### Se lee el indice all-cambium-device-client

* En este indice se guarda el detalle de los radio por fecha<br>
* Detalle conexiones cruza con device clients. Con estos se calculan los totales por marca

In[224]:

In [ ]:
def traeRadio(fecha_max,fecha_tope):
    total_docs = 100000
    response = es.search(
        index= 'all-'+parametros.cambium_d_c_index,
        body={
            "_source": ['mac', 'ap_mac', 'radio.band', 'radio.rx_bytes', 'radio.tx_bytes','fecha_control']
              , "query": {
                  "range": {
                    "fecha_control": {
                      "gte": fecha_max_mintic,
                      "lt": fecha_tope_mintic
                      #"gte": "2021-05-26 15:00:00",
                      #"lt": "2021-05-26 15:10:00"  
                    }
                  }
              }
        },
        size=total_docs
    )
    elastic_docs = response["hits"]["hits"]
    # fields = {}
    # for num, doc in enumerate(elastic_docs):
    #     source_data = doc["_source"]
    #     for key, val in source_data.items():
    #         try:
    #             fields[key] = np.append(fields[key], val)
    #         except KeyError:
    #             fields[key] = np.array([val])

    # datos_radio = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in fields.items() ]))

    return pd.DataFrame([x["_source"] for x in elastic_docs])

total_docs = 100000
response = es.search(
    index= 'all-'+parametros.cambium_d_c_index,
    body={
        "_source": ['mac', 'ap_mac', 'radio.band', 'radio.rx_bytes', 'radio.tx_bytes','fecha_control']
          , "query": {
              "range": {
                "fecha_control": {
                  "gte": fecha_max_mintic,
                  "lt": fecha_tope_mintic
                  #"gte": "2021-05-26 15:00:00",
                  #"lt": "2021-05-26 15:10:00"  
                }
              }
          }
    },
    size=total_docs
)
elastic_docs = response["hits"]["hits"]
# fields = {}
# for num, doc in enumerate(elastic_docs):
#     source_data = doc["_source"]
#     for key, val in source_data.items():
#         try:
#             fields[key] = np.append(fields[key], val)
#         except KeyError:
#             fields[key] = np.array([val])

# datos_radio = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in fields.items() ]))

datos_radio = pd.DataFrame([x["_source"] for x in elastic_docs])

datos_performance = datos_radio

In[225]:

In [ ]:
if datos_performance is None or datos_performance.empty:
    while (datos_performance is None or datos_performance.empty) and ((datetime.strptime(fecha_max_mintic[0:10], '%Y-%m-%d').strftime("%Y-%m-%d %H:%M:%S")) < str(now.strftime("%Y-%m-%d %H:%M:%S"))):
        fecha_max_mintic = (datetime.strptime(fecha_max_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(minutes=50)).strftime("%Y-%m-%d %H:%M:%S")
        fecha_tope_mintic = (datetime.strptime(fecha_tope_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(minutes=50)).strftime("%Y-%m-%d %H:%M:%S")
        datos_performance = traeRadio(fecha_max_mintic,fecha_tope_mintic)
else:
    pass

In[226]:

In [ ]:
try:
    datos_logins = datos_det_conex[['fecha_control', 'site_id', 'documento','usuarios.macRed']].groupby(["fecha_control","site_id","usuarios.macRed"])['documento'].nunique().reset_index()
    datos_logins= datos_logins.rename(columns={'documento' : 'usuarios.sesiones_Usuarios'})
    
    ################# datos_performance #####################
    datos_performance['fecha_control'] = datos_performance["fecha_control"].str[0:-4] + '0:00'
    datos_performance.replace('','0',inplace=True)
    datos_performance.fillna({'radio.rx_bytes':0,'radio.tx_bytes':0},inplace=True)
    datos_performance['radio.rx_bytes'] = datos_performance['radio.rx_bytes'].astype(float)
    datos_performance['radio.tx_bytes'] = datos_performance['radio.tx_bytes'].astype(float)
    
    datos_performance = datos_performance.groupby(['ap_mac', 'fecha_control', 'mac', 'radio.band']).agg(['max']).reset_index()
    datos_performance.columns = datos_performance.columns.droplevel(1)
    datos_performance = datos_performance[['ap_mac'
                                         , 'fecha_control'
                                         , 'radio.rx_bytes'
                                         , 'radio.tx_bytes']].groupby(['ap_mac'
                                                                     , 'fecha_control']).agg(['sum']).reset_index()
    datos_performance.columns = datos_performance.columns.droplevel(1)
    datos_performance = datos_performance.rename(columns={'radio.rx_bytes': 'usuarios.consumoUsuariosDescarga_aux'
                                                     , 'radio.tx_bytes':'usuarios.consumoUsuariosCarga_aux'})
    
    datos_performance['usuarios.consumoUsuariosDescarga'] = round((datos_performance['usuarios.consumoUsuariosDescarga_aux']/float(1<<30)),6)
    datos_performance['usuarios.consumoUsuariosCarga'] = round((datos_performance['usuarios.consumoUsuariosCarga_aux']/float(1<<30)),6)
    datos_performance['usuarios.consumoUsuarios'] = datos_performance['usuarios.consumoUsuariosDescarga'] + datos_performance['usuarios.consumoUsuariosCarga']
    datos_performance['usuarios.consumoUsuarios'] = round(datos_performance['usuarios.consumoUsuarios'],6)
    datos_performance = datos_performance.rename(columns={'ap_mac':'usuarios.macRed'})
    
    
    datos_performance = pd.merge(datos_performance,datos_dev[['site_id','usuarios.macRed','usuarios.apGroup']],on='usuarios.macRed', how='left')
    
    aux_performance = pd.merge(datos_performance,  datos_semilla, on='site_id',how='inner')
    aux_performance.fillna({'usuarios.consumoUsuariosDescarga':0,
                              'usuarios.consumoUsuariosCarga':0,
                              'usuarios.consumoUsuarios':0,
                              'usuarios.apGroup':'No identificado'},inplace=True)
    aux_performance = aux_performance.rename(columns={'fecha_control':'usuarios.fechaControl'
                                                           ,'site_id' : 'usuarios.siteID'})
    
    aux_performance["usuarios.fecha"] = aux_performance["usuarios.fechaControl"].str.split(" ", n = 1, expand = True)[0]
    aux_performance["usuarios.anyo"] = aux_performance["usuarios.fecha"].str[0:4]
    aux_performance["usuarios.mes"] = aux_performance["usuarios.fecha"].str[5:7]
    aux_performance["usuarios.dia"] = aux_performance["usuarios.fecha"].str[8:10]
    aux_performance["usuarios.hora"] = aux_performance["usuarios.fechaControl"].str.split(" ", n = 1, expand = True)[1].str.split(":", n = 2, expand = True)[0]
    aux_performance["usuarios.minuto"] = aux_performance["usuarios.fechaControl"].str.split(" ", n = 1, expand = True)[1].str.split(":", n = 2, expand = True)[1]
    
    aux_performance['nombreDepartamento'] = aux_performance['usuarios.nombreDepartamento']
    aux_performance['nombreMunicipio'] = aux_performance['usuarios.nombreMunicipio']
    aux_performance['idBeneficiario'] = aux_performance['usuarios.idBeneficiario']
    aux_performance['fecha'] = aux_performance['usuarios.fecha']
    aux_performance['anyo'] = aux_performance['usuarios.anyo']
    aux_performance['mes'] = aux_performance['usuarios.mes']
    aux_performance['dia'] = aux_performance['usuarios.dia']
    
    
    ##################
    usuariosConectados = datos_det_conex[["fecha_control","site_id","usuarios.macRed","mac_usuario"]].groupby(["fecha_control","site_id","usuarios.macRed"]).agg(['count']).reset_index()
    usuariosConectados.columns = usuariosConectados.columns.droplevel(1)
    usuariosConectados= usuariosConectados.rename(columns={'mac_usuario' : 'usuarios.tablero11usuariosConectados'})
    
    ###################
    usuariosConectados = pd.merge(usuariosConectados,datos_logins, on=["fecha_control","site_id","usuarios.macRed"], how='inner')
    
    usuariosConectados= usuariosConectados.rename(columns={'usuarios.sesiones_Usuarios' : 'usuarios.tablero11sesiones_Usuarios'})
    
    ###################
    usuariosConectados = pd.merge(datos_semilla,  usuariosConectados, on=['site_id'], how='inner')
    
   
    #Al parecer no esta viniendo consumoUsuario....
    
    usuariosConectados.fillna({'usuarios.consumoUsuarios' : 0
                              ,'usuarios.consumoUsuariosDescarga':0
                              ,'usuarios.consumoUsuariosCarga':0},inplace=True)
    
    ####################
    usuariosConectados = pd.merge(usuariosConectados,datos_dev[['usuarios.apGroup','usuarios.macRed']], on ='usuarios.macRed', how='left')
    
    usuariosConectados.fillna({'usuarios.apGroup':'No identificado'},inplace=True)
    usuariosConectados['usuarios.tablero11usuariosConectados'] = usuariosConectados['usuarios.tablero11usuariosConectados'].astype(int)
    usuariosConectados = usuariosConectados.rename(columns={'fecha_control':'usuarios.fechaControl'
                                                           ,'site_id' : 'usuarios.siteID'})
    
    usuariosConectados["usuarios.fecha"] = usuariosConectados["usuarios.fechaControl"].str.split(" ", n = 1, expand = True)[0]
    usuariosConectados["usuarios.anyo"] = usuariosConectados["usuarios.fecha"].str[0:4]
    usuariosConectados["usuarios.mes"] = usuariosConectados["usuarios.fecha"].str[5:7]
    usuariosConectados["usuarios.dia"] = usuariosConectados["usuarios.fecha"].str[8:10]
    usuariosConectados["usuarios.hora"] = usuariosConectados["usuarios.fechaControl"].str.split(" ", n = 1, expand = True)[1].str.split(":", n = 2, expand = True)[0]
    usuariosConectados["usuarios.minuto"] = usuariosConectados["usuarios.fechaControl"].str.split(" ", n = 1, expand = True)[1].str.split(":", n = 2, expand = True)[1]
    
    usuariosConectados['nombreDepartamento'] = usuariosConectados['usuarios.nombreDepartamento']
    usuariosConectados['nombreMunicipio'] = usuariosConectados['usuarios.nombreMunicipio']
    usuariosConectados['idBeneficiario'] = usuariosConectados['usuarios.idBeneficiario']
    usuariosConectados['fecha'] = usuariosConectados['usuarios.fecha']
    usuariosConectados['anyo'] = usuariosConectados['usuarios.anyo']
    usuariosConectados['mes'] = usuariosConectados['usuarios.mes']
    usuariosConectados['dia'] = usuariosConectados['usuarios.dia']
    
    #####################
    columnas_aux_performance=['usuarios.macRed', 'usuarios.fechaControl',                               'usuarios.consumoUsuariosDescarga_aux',
                               'usuarios.consumoUsuariosCarga_aux', 'usuarios.consumoUsuariosDescarga',\
                               'usuarios.consumoUsuariosCarga', 'usuarios.consumoUsuarios',\
                               'usuarios.siteID', 'usuarios.apGroup']
    columnas_merge = ['usuarios.fechaControl','usuarios.siteID','usuarios.apGroup','usuarios.macRed']
    
    usuariosConectados=pd.merge(usuariosConectados,aux_performance[columnas_aux_performance],on=columnas_merge, how='left')
    
    usuariosConectados = usuariosConectados.rename(columns=
                                                   {'usuarios.consumoUsuariosCarga_aux':'usuarios.tablero11consumoUsuariosCarga_aux'
                                                    ,'usuarios.consumoUsuariosDescarga_aux':'usuarios.tablero11consumoUsuariosDescarga_aux'
                                                    ,'usuarios.consumoUsuariosCarga' : 'usuarios.tablero11consumoUsuariosCarga'
                                                    ,'usuarios.consumoUsuariosDescarga':'usuarios.tablero11consumoUsuariosDescarga'
                                                    ,'usuarios.consumoUsuarios':'usuarios.tablero11consumoUsuarios'
                                                   })
    
except:
    pass

In[227]:

In [ ]:
try:
    datos_consumos = datos_consumos.rename(columns={'lugar_cod' :'site_id'})
    datos_consumos['mac_ap'] = datos_consumos['mac_ap'].str.replace('-',':')
    datos_consumos['fecha_control'] = datos_consumos["fecha_inicio"].str[0:-4] + '0:00'
    
    tiempoPromedioSesionSitio=datos_consumos[['site_id','mac_ap','fecha_control','tiempo_sesion_minutos']].groupby(['site_id','mac_ap','fecha_control']).agg(['mean']).reset_index()
    tiempoPromedioSesionSitio.columns = tiempoPromedioSesionSitio.columns.droplevel(1)
    tiempoPromedioSesionSitio = tiempoPromedioSesionSitio.rename(columns={'tiempo_sesion_minutos' : 'usuarios.tiempoPromedioSesionSitio'})
    tiempoPromedioSesionSitio['usuarios.tiempoPromedioSesionSitio'] = round(tiempoPromedioSesionSitio['usuarios.tiempoPromedioSesionSitio'],6)
    tiempoPromedioSesionSitio = tiempoPromedioSesionSitio.rename(columns={'mac_ap' : 'usuarios.macRed'
                                                                     , 'fecha_control' : 'usuarios.fechaControl'})
    
    
    datos_dev.rename(columns={'usuarios.siteID':'site_id'},inplace=True)
    
    tiempoPromedioSesionSitio = pd.merge(tiempoPromedioSesionSitio,datos_semilla, on ='site_id', how='inner')
    tiempoPromedioSesionSitio = pd.merge(tiempoPromedioSesionSitio, datos_dev, on=['site_id','usuarios.macRed'], how='left')
    tiempoPromedioSesionSitio.fillna({'usuarios.apGroup':'No identificado'},inplace=True)
    tiempoPromedioSesionSitio = tiempoPromedioSesionSitio.rename(columns={'site_id' : 'usuarios.siteID'})
    tiempoPromedioSesionSitio["usuarios.fecha"] = tiempoPromedioSesionSitio["usuarios.fechaControl"].str[0:10]
    tiempoPromedioSesionSitio["usuarios.anyo"] = tiempoPromedioSesionSitio["usuarios.fecha"].str[0:4]
    tiempoPromedioSesionSitio["usuarios.mes"] = tiempoPromedioSesionSitio["usuarios.fecha"].str[5:7]
#     tiempoPromedioSesionSitio["usuarios.dia"] = tiempoPromedioSesionSitio["usuarios.fecha"].str[8:10]
#     tiempoPromedioSesionSitio["usuarios.hora"] = tiempoPromedioSesionSitio["usuarios.fechaControl"].str.split(" ", n = 1, expand = True)[1].str.split(":", n = 2, expand = True)[0]["usuarios.hora"] = tiempoPromedioSesionSitio["usuarios.fechaControl"].str.split(" ", n = 1, expand = True)[1].str.split(":", n = 2, expand = True)[0]
#     tiempoPromedioSesionSitio["usuarios.minuto"] = tiempoPromedioSesionSitio["usuarios.fechaControl"].str.split(" ", n = 1, expand = True)[1].str.split(":", n = 2, expand = True)[1]
    
    tiempoPromedioSesionSitio['nombreDepartamento'] = tiempoPromedioSesionSitio['usuarios.nombreDepartamento']
    tiempoPromedioSesionSitio['nombreMunicipio'] = tiempoPromedioSesionSitio['usuarios.nombreMunicipio']
    tiempoPromedioSesionSitio['idBeneficiario'] = tiempoPromedioSesionSitio['usuarios.idBeneficiario']
    tiempoPromedioSesionSitio['fecha'] = tiempoPromedioSesionSitio['usuarios.fecha']
    
    tiempoPromedioSesionSitio['@timestamp'] = now.isoformat()
    
#     salida = helpers.bulk(es, doc_generator(tiempoPromedioSesionSitio))
#     print("Fecha: ", now,"- Tiempo promedio sesion en sitio insertado en indice principal:",salida[0])
except:
#     print("Fecha: ", now,"- Nada para insertar en indice principal")
    pass

# Uso servicio de internet insertados en indice principal

la lista use_these_keys se usa para referenciar cuales campos del dataframe irÃ¡n al indice final. si los datos no se declaran en este, no se insertarÃ¡n

In[228]:

use_these_keys = ['usuarios.siteID'<br>
                  ,'usuarios.tablero11.nomCentroDigital'<br>
                  , 'usuarios.tablero11.localidad'<br>
                  , 'usuarios.tablero11.nombreDepartamento'<br>
                  , 'usuarios.tablero11.nombreMunicipio'<br>
                  , 'usuarios.tablero11.idBeneficiario'<br>
                  , 'usuarios.tablero11.location'<br>
#                   , 'usuarios.tablero11sesiones_Usuarios'<br>
#                   , 'usuarios.tablero11consumoUsuarios'<br>
                  , 'usuarios.tablero11.apGroup'<br>
                  , 'usuarios.tablero11.fechaControl'<br>
                  , 'usuarios.tablero11.fecha'<br>
                  , 'usuarios.tablero11.anyo'<br>
                  , 'usuarios.tablero11.mes'<br>
                  , '@timestamp']

def doc_generator(df,use_these_keys):<br>
    df_iter = df.iterrows()<br>
    for index, document in df_iter:<br>
        yield {<br>
                "_index": indice, <br>
                "_id": f"{ 'UsoServicio-' + str(document['usuarios.macRed']) + '-' + str(document['usuarios.detallesTecnologiasTerminales']) + '-' + str(document['usuarios.fechaControl'])}",<br>
                "_source": filterKeys(document,use_these_keys),<br>
            }

### Asociando datos de Speed test

Se tiene una lectura diara de velocidad para cada centro. Por tanto se debe cruzar con el fjulo principal, haciendo uso solo del aÃ±o, mes dÃ­a, sin incluir la hora.

In[229]:

In [ ]:
def traeVelocidad(fecha_max_mintic,fecha_tope_mintic):
    total_docs = 10000
    response = es.search(
        index= parametros.speed_index+'*',
        body={
                "_source": ["beneficiary_code","locationid", "result_start_date"
                            , "result_download_mbps", "result_upload_mbps"],
                "query": {
                    "range": {
                        "result_start_date": {
                            "gte": fecha_max_mintic.split(' ')[0]+'T00:00:00',
                            "lt": fecha_tope_mintic.split(' ')[0]+'T23:59:59'
                        }
                    }
                }
        },
        size=total_docs
    )
    #print(es.info())
    elastic_docs = response["hits"]["hits"]
    fields = {}
    for num, doc in enumerate(elastic_docs):
        source_data = doc["_source"]
        for key, val in source_data.items():
            try:
                fields[key] = np.append(fields[key], val)
            except KeyError:
                fields[key] = np.array([val])
    return pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in fields.items() ]))

* Se genera fecha en yyyy-mm-dd, y cada campo por separado<br>
* La hora y minuto se toma aparte<br>
<br>
Valores que se convierten a cero si son nulos<br>
* trafico.anchoBandaDescarga<br>
* trafico.anchoBandaCarga

In[230]:

In[231]:

In [ ]:
fecha_max_mintic = fecha_ejecucion
fecha_tope_mintic = (datetime.strptime(fecha_max_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(minutes=50)-timedelta(seconds=1)).strftime("%Y-%m-%d %H:%M:%S")
datos_speed = traeVelocidad(fecha_max_mintic,fecha_tope_mintic)

if datos_speed is None or datos_speed.empty:
    while (datos_speed is None or datos_speed.empty) and ((datetime.strptime(fecha_max_mintic[0:10], '%Y-%m-%d').strftime("%Y-%m-%d %H:%M:%S")) < str(now.strftime("%Y-%m-%d %H:%M:%S"))):
        fecha_max_mintic = (datetime.strptime(fecha_max_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(minutes=50)).strftime("%Y-%m-%d %H:%M:%S")
        fecha_tope_mintic = (datetime.strptime(fecha_tope_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(minutes=50)).strftime("%Y-%m-%d %H:%M:%S")
        datos_speed = traeVelocidad(fecha_max_mintic,fecha_tope_mintic)
else:
    pass

In [ ]:
# In[231]:


try:
    datos_speed = datos_speed.drop(datos_speed[(datos_speed["result_download_mbps"]<0) | (datos_speed["result_upload_mbps"]<0)].index)
    datos_speed['trafico.totales.fecha'] = datos_speed['result_start_date'].str.split("T", n = 1, expand = True)[0]
    datos_speed['result_download_mbps'] = datos_speed['result_download_mbps'] * 1000
    datos_speed['result_upload_mbps'] = datos_speed['result_upload_mbps'] * 1000
    datos_speed = datos_speed[['beneficiary_code','trafico.totales.fecha','result_download_mbps','result_upload_mbps']].groupby(['beneficiary_code','trafico.totales.fecha']).agg(['max']).reset_index()
    datos_speed.columns = datos_speed.columns.droplevel(1)
    datos_speed.rename(columns={'result_download_mbps': 'trafico.anchoBandaDescarga'
                             ,'result_upload_mbps' :  'trafico.anchoBandaCarga'
                             , 'beneficiary_code' : 'site_id'
                             }, inplace=True)

    datos_speed["trafico.totales.anyo"] = datos_speed["trafico.totales.fecha"].str[0:4]
    datos_speed["trafico.totales.mes"] = datos_speed["trafico.totales.fecha"].str[5:7]
    datos_speed["trafico.totales.dia"] = datos_speed["trafico.totales.fecha"].str[8:10]
    datos_speed = pd.merge(datos_speed,  datos_semilla, on='site_id', how='inner')
    datos_speed = datos_speed.rename(columns={'site_id' : 'trafico.siteID'})
    datos_speed.dropna(subset=['trafico.anchoBandaDescarga','trafico.anchoBandaCarga'])
    datos_speed.fillna({'trafico.anchoBandaDescarga':0
                      , 'trafico.anchoBandaCarga':0
                       },inplace=True)
    datos_speed.fillna('', inplace=True)
    datos_speed['nombreDepartamento'] = datos_speed['trafico.nombreDepartamento']
    datos_speed['nombreMunicipio'] = datos_speed['trafico.nombreMunicipio']
    datos_speed['idBeneficiario'] = datos_speed['trafico.idBeneficiario']
    datos_speed['fecha'] = datos_speed['trafico.totales.fecha']
    datos_speed['anyo'] = datos_speed['trafico.totales.anyo']
    datos_speed['mes'] = datos_speed['trafico.totales.mes']
    datos_speed['dia'] = datos_speed['trafico.totales.dia']
    datos_speed['@timestamp'] = now.isoformat()

    datos_speed.rename(columns={
       'trafico.siteID':'usuarios.siteID',
       'trafico.nombreDepartamento':'usuarios.nombreDepartamento',
       'trafico.codISO':'usuarios.codISO',
       'trafico.sistemaEnergia':'usuarios.sistemaEnergia',
       'trafico.nombreMunicipio':'usuarios.nombreMunicipio',
       'trafico.localidad' : 'usuarios.localidad',
       'trafico.nomCentroDigital' : 'usuarios.nomCentroDigital',
       'trafico.idBeneficiario':'usuarios.idBeneficiario',
       'trafico.location':'usuarios.location',
       'trafico.anchoBandaDescarga':'usuarios.anchoBandaDescarga',
       'trafico.anchoBandaCarga':'usuarios.anchoBandaCarga',
       'trafico.totales.fecha':'usuarios.fecha',
       'trafico.totales.anyo':'usuarios.anyo',
       'trafico.totales.mes':'usuarios.mes',
       'trafico.totales.dia':'usuarios.dia' }, inplace=True)
        
#     salida = helpers.bulk(es, doc_generator(datos_speed))
#     print("Fecha: ", now,"- Datos Velocidad carga y descarga (Trafico) en indice principal:",salida[0])
except:
#     print("Fecha: ", now,"- No hay Datos Velocidad carga y descarga(Trafico) para insertar en indice principal")
    pass

In[232]:

O se utiliza informacion del uso del servicio de Internet por MAC address en el Tablero 11.

In [ ]:
try:    
    datos_radio['fecha_control'] = datos_radio['fecha_control'].str[0:15]+'0:00'
    datos_radio = datos_radio.groupby(['ap_mac', 'fecha_control', 'mac', 'radio.band']).agg(['max']).reset_index()
    datos_radio.columns = datos_radio.columns.droplevel(1)
    datos_radio = datos_radio[['ap_mac', 'fecha_control', 'radio.band', 'radio.rx_bytes',
           'radio.tx_bytes']].groupby(['ap_mac', 'fecha_control', 'radio.band']).agg(['sum']).reset_index()
    datos_radio.columns = datos_radio.columns.droplevel(1)
    datos_radio = datos_radio.rename(columns={'ap_mac' : 'usuarios.macRed'})
    
    usoServicioInternetSitio = pd.merge(datos_radio,datos_dev[['usuarios.apGroup', 'site_id', 'usuarios.macRed']], on ='usuarios.macRed', how='inner')
    usoServicioInternetSitio.fillna({'radio.rx_bytes':0,'radio.tx_bytes':0},inplace=True)
    usoServicioInternetSitio.fillna({'usuarios.macRed':'','radio.band':'No identificado','usuarios.apGroup':'No identificado'},inplace=True)
    usoServicioInternetSitio['usuarios.usoServicioInternetSitio'] = usoServicioInternetSitio['radio.rx_bytes'].astype(float) + usoServicioInternetSitio['radio.tx_bytes'].astype(float)
    usoServicioInternetSitio = pd.merge(usoServicioInternetSitio,datos_semilla, on ='site_id', how='inner')
    usoServicioInternetSitio= usoServicioInternetSitio.rename(columns={'site_id' : 'usuarios.siteID'
                                                                    ,'radio.band' : 'usuarios.detallesTecnologiasTerminales'
                                                                    , 'fecha_control' : 'usuarios.fechaControl'})
    usoServicioInternetSitio['usuarios.usoServicioInternetSitio'] = round((usoServicioInternetSitio['usuarios.usoServicioInternetSitio']/float(1<<30)),6) 
    usoServicioInternetSitio["usuarios.fecha"] = usoServicioInternetSitio["usuarios.fechaControl"].str.split(" ", n = 1, expand = True)[0]
    usoServicioInternetSitio["usuarios.anyo"] = usoServicioInternetSitio["usuarios.fecha"].str[0:4]
    usoServicioInternetSitio["usuarios.mes"] = usoServicioInternetSitio["usuarios.fecha"].str[5:7]
    usoServicioInternetSitio["usuarios.dia"] = usoServicioInternetSitio["usuarios.fecha"].str[8:10]
    usoServicioInternetSitio["usuarios.hora"] = usoServicioInternetSitio["usuarios.fechaControl"].str.split(" ", n = 1, expand = True)[1].str.split(":", n = 2, expand = True)[0]
    usoServicioInternetSitio["usuarios.minuto"] = usoServicioInternetSitio["usuarios.fechaControl"].str.split(" ", n = 1, expand = True)[1].str.split(":", n = 2, expand = True)[1]
    
    usoServicioInternetSitio['nombreDepartamento'] = usoServicioInternetSitio['usuarios.nombreDepartamento']
    usoServicioInternetSitio['nombreMunicipio'] = usoServicioInternetSitio['usuarios.nombreMunicipio']
    usoServicioInternetSitio['idBeneficiario'] = usoServicioInternetSitio['usuarios.idBeneficiario']
    usoServicioInternetSitio['fecha'] = usoServicioInternetSitio['usuarios.fecha']
    usoServicioInternetSitio['anyo'] = usoServicioInternetSitio['usuarios.anyo']
    usoServicioInternetSitio['mes'] = usoServicioInternetSitio['usuarios.mes']
    usoServicioInternetSitio['dia'] = usoServicioInternetSitio['usuarios.dia']
    
    usoServicioInternetSitio['@timestamp'] = now.isoformat()
    
    #Cambio de variables utilizando los campos del Tablero...
    
#     usoServicioInternetSitio=cambioVariable(usoServicioInternetSitio)
    
    #Se comenta escritura en el indice porque no se necesita esa estructura
    #salida = helpers.bulk(es, doc_generator(usoServicioInternetSitio))
    #print("Fecha: ", now,"- Uso servicio y tecnologÃ­a terminal insertados en indice principal:",salida[0])
except:
#     print("Fecha: ", now,"- Ningun uservicio/tecnologÃ­a terminal conectado para insertar en indice principal")
    pass

* se toma el maximo radio para cada mac de dispositivo final<br>
* se suma agrupando por ap_mac, banda y fecha<br>
* El resultado es un total para cada banda, fecha y ap_mac

Se renombra fecha_control para cruzar

In[233]:

In [ ]:
try:
    use_these_keys = [ 'usuarios.tablero09.tiempoPromedioSesionSitio'
                      , 'usuarios.tablero09.siteID'
                      ,'usuarios.tablero09.nomCentroDigital'
                      , 'usuarios.tablero09.localidad'
                      , 'usuarios.tablero09.nombreDepartamento'
                      , 'usuarios.tablero09.nombreMunicipio'
                      , 'usuarios.tablero09.idBeneficiario'
                      , 'usuarios.tablero09.location'
                      , 'usuarios.tablero09.sesiones_Usuarios'
                      , 'usuarios.tablero09.usuariosConectados'
                      , 'usuarios.tablero09.consumoUsuarios'
#                       , 'usuarios.tablero09.consumoUsuariosDescarga_aux'
#                       , 'usuarios.tablero09.consumoUsuariosCarga_aux'
                      , 'usuarios.tablero09.consumoUsuariosDescarga'
                      , 'usuarios.tablero09.consumoUsuariosCarga'
                      , 'usuarios.tablero09.apGroup'
                      , 'usuarios.tablero09.fechaControl'
                      , 'usuarios.tablero09.fecha'
                      , 'usuarios.tablero09.anyo'
                      , 'usuarios.tablero09.mes'
                      , 'usuarios.tablero09.macRed'
                      , 'usuarios.tablero09.usoServicioInternetSitio'
                      , 'usuarios.tablero09.anchoBandaDescarga'
                      , 'usuarios.tablero09.anchoBandaCarga'
                      , '@timestamp']


    usuariosConectados['@timestamp'] = now.isoformat()
    
    
    #Seleccionar campos del Tablero..
    
    #Cambio de variables en usuariosConectados...
    
    usuariosConectados = usuariosConectados[[  'usuarios.nomCentroDigital',  
                                               'usuarios.localidad',
                                               'usuarios.siteID', 
                                               'usuarios.nombreDepartamento',
                                               'usuarios.nombreMunicipio',
                                               'usuarios.idBeneficiario', 
                                               'usuarios.location', 
                                               'usuarios.fechaControl',
                                               'usuarios.macRed', 
                                               'usuarios.tablero11usuariosConectados',
#                                                'usuarios.tablero09.tiempoPromedioSesionSitio',
                                               'usuarios.tablero11sesiones_Usuarios', 
                                               'usuarios.apGroup',
                                               'usuarios.fecha', 
                                               'usuarios.anyo', 
                                               'usuarios.mes',
                                               'usuarios.tablero11consumoUsuariosDescarga',
                                               'usuarios.tablero11consumoUsuariosCarga',
                                               'usuarios.tablero11consumoUsuarios',
                                               '@timestamp']]
    
    usuariosConectados = pd.merge(usuariosConectados,
                                 usoServicioInternetSitio[['usuarios.siteID',
                                                           'usuarios.macRed',
                                                           'usuarios.fechaControl',
                                                           'usuarios.usoServicioInternetSitio']]
                                 ,on=['usuarios.siteID','usuarios.macRed','usuarios.fechaControl'],
                                  how='left')
    
#     usuariosConectados = cambioVariable(usuariosConectados)

    datos_speed = datos_speed[['trafico.siteID', 
                               'trafico.totales.fecha', 
                               'trafico.anchoBandaDescarga',
                               'trafico.anchoBandaCarga']]
    
    datos_speed.columns = ['usuarios.siteID', 
                           'usuarios.fecha', 
                           'usuarios.anchoBandaDescarga',
                           'usuarios.anchoBandaCarga']
    
    usuariosConectados =  pd.merge(usuariosConectados,datos_speed,on=['usuarios.siteID','usuarios.fecha'],how='left')
    
    usuariosConectados =  pd.merge(usuariosConectados,
                          tiempoPromedioSesionSitio[['usuarios.siteID','usuarios.fecha','usuarios.tiempoPromedioSesionSitio']],
                          on=['usuarios.siteID','usuarios.fecha'],how='left')
    
    usuariosConectados = usuariosConectados.rename(columns={
                                                 'usuarios.nomCentroDigital':'usuarios.tablero09.nomCentroDigital',
                                                 'usuarios.localidad':'usuarios.tablero09.localidad', 
                                                 'usuarios.siteID':'usuarios.tablero09.siteID',
                                                 'usuarios.nombreDepartamento':'usuarios.tablero09.nombreDepartamento',
                                                 'usuarios.nombreMunicipio':'usuarios.tablero09.nombreMunicipio',
                                                 'usuarios.idBeneficiario':'usuarios.tablero09.idBeneficiario',
                                                 'usuarios.location':'usuarios.tablero09.location',
                                                 'usuarios.fechaControl':'usuarios.tablero09.fechaControl',
                                                 'usuarios.macRed':'usuarios.tablero09.macRed',
                                                 'usuarios.tablero11usuariosConectados':'usuarios.tablero09.usuariosConectados',
                                                 'usuarios.tablero11sesiones_Usuarios':'usuarios.tablero09.sesiones_Usuarios',
                                                 'usuarios.apGroup':'usuarios.tablero09.apGroup',
                                                 'usuarios.fecha':'usuarios.tablero09.fecha', 
                                                 'usuarios.anyo':'usuarios.tablero09.anyo', 
                                                 'usuarios.mes':'usuarios.tablero09.mes',
                                                 'usuarios.tablero11consumoUsuariosDescarga':'usuarios.tablero09.consumoUsuariosDescarga',
                                                 'usuarios.tablero11consumoUsuariosCarga':'usuarios.tablero09.consumoUsuariosCarga',
                                                 'usuarios.tablero11consumoUsuarios':'usuarios.tablero09.consumoUsuarios',
                                                 'usuarios.usoServicioInternetSitio':'usuarios.tablero09.usoServicioInternetSitio',
                                                 'usuarios.anchoBandaDescarga':'usuarios.tablero09.anchoBandaDescarga',
                                                 'usuarios.anchoBandaCarga':'usuarios.tablero09.anchoBandaCarga',
                                                 'usuarios.tiempoPromedioSesionSitio':'usuarios.tablero09.tiempoPromedioSesionSitio'})
    
    
    def doc_generator_consumo(df,use_these_keys):
        df_iter = df.iterrows()
        for index, document in df_iter:
            yield {
                    "_index": indice, 
                    "_id": f"{ 'usuariosConectados-'+ str(document['usuarios.tablero09.siteID']) + str(document['usuarios.tablero09.macRed']) + '-' + str(document['usuarios.tablero09.fechaControl'])+'-'+str(random.randrange(10000))}",
                    "_source": filterKeys(document,use_these_keys),
                }

    #Se comenta escritura en el indice porque no se necesita esa estructura#Se comenta escritura en el indice porque no se necesita esa estructura            }
    
#     usuariosConectados.fillna("",inplace=True)

    usuariosConectados.fillna({'usuarios.tablero09.anchoBandaDescarga':0
                              ,'usuarios.tablero09.anchoBandaCarga':0
                              ,'usuarios.tablero09.usoServicioInternetSitio':0
                              ,'usuarios.tablero09.consumoUsuarios':0
                              ,'usuarios.tablero09.consumoUsuariosCarga':0
                              ,'usuarios.tablero09.consumoUsuariosDescarga':0
                              ,"usuarios.tablero09.tiempoPromedioSesionSitio":0
                              },inplace=True)
    
    usuariosConectados.fillna("",inplace=True)
    
    salida = helpers.bulk(es, doc_generator_consumo(usuariosConectados,use_these_keys))
    
    print("Fecha: ", now,"- Usuarios conectados insertados en indice principal:",salida[0])
    
except Exception as e:  
    
    print("Fecha: ", now,"- Ningun usuario conectado para insertar en indice principal:")

## Totalizar la cantidad de dispositivos por usuario

* Para contabilizar los dispositivos por usuario, se debe contar la cantidad de ocurrencias de mac_usuario dentro de detalle de conexiones, agrupando por tipo documento y documento. <br>
* AÃºn cuando la mayoria de las fuentes referencia a mac_usuario, no se debe confundir a la hora de agrupar con los datos reales de usuario que en este caso serÃ­an el documento de identidad como campo de identificaciÃ³n

In[234]:

NO se utiliza campos de CONTEO DE DISPOSITIVOS..

try:<br>
    conteoDispositivos = datos_det_conex[['fecha_control','site_id', 'documento', 'tipodoc','fecha']].groupby(['fecha_control','site_id', 'documento', 'tipodoc']).agg(['count']).reset_index()<br>
    conteoDispositivos.columns = conteoDispositivos.columns.droplevel(1)<br>
    conteoDispositivos= conteoDispositivos.rename(columns={'fecha' : 'conteo'})<br>
    conteoDispositivos = conteoDispositivos[['fecha_control','site_id','conteo']].groupby(['fecha_control','site_id']).agg(['sum']).reset_index()<br>
    conteoDispositivos.columns = conteoDispositivos.columns.droplevel(1)<br>
    conteoDispositivos= conteoDispositivos.rename(columns={'conteo' : 'usuarios.conteoDispositivos'})<br>
    conteoDispositivos = pd.merge(conteoDispositivos, datos_semilla, on='site_id', how='inner')<br>
    conteoDispositivos = conteoDispositivos.rename(columns={'fecha_control' : 'usuarios.fechaControl'})<br>
    conteoDispositivos["usuarios.fecha"] = conteoDispositivos["usuarios.fechaControl"].str.split(" ", n = 1, expand = True)[0]<br>
    conteoDispositivos["usuarios.anyo"] = conteoDispositivos["usuarios.fecha"].str[0:4]<br>
    conteoDispositivos["usuarios.mes"] = conteoDispositivos["usuarios.fecha"].str[5:7]<br>
    conteoDispositivos["usuarios.dia"] = conteoDispositivos["usuarios.fecha"].str[8:10]<br>
    conteoDispositivos["usuarios.hora"] = conteoDispositivos["usuarios.fechaControl"].str.split(" ", n = 1, expand = True)[1].str.split(":", n = 2, expand = True)[0]<br>
    conteoDispositivos["usuarios.minuto"] = conteoDispositivos["usuarios.fechaControl"].str.split(" ", n = 1, expand = True)[1].str.split(":", n = 2, expand = True)[1]<br>
    conteoDispositivos= conteoDispositivos.rename(columns={'site_id' : 'usuarios.siteID'})

In [ ]:
    
#     conteoDispositivos['nombreDepartamento'] = conteoDispositivos['usuarios.nombreDepartamento']
#     conteoDispositivos['nombreMunicipio'] = conteoDispositivos['usuarios.nombreMunicipio']
#     conteoDispositivos['idBeneficiario'] = conteoDispositivos['usuarios.idBeneficiario']
#     conteoDispositivos['fecha'] = conteoDispositivos['usuarios.fecha']
#     conteoDispositivos['anyo'] = conteoDispositivos['usuarios.anyo']
#     conteoDispositivos['mes'] = conteoDispositivos['usuarios.mes']
#     conteoDispositivos['dia'] = conteoDispositivos['usuarios.dia']
# except:
#     pass

# 4. Insertando promedio dispositivos en indice principal

la lista use_these_keys se usa para referenciar cuales campos del dataframe irÃ¡n al indice final. si los datos no se declaran en este, no se insertarÃ¡n

In[235]:

NO se utiliza campos de CONTEO DE DISPOSITIVOS..

try:<br>
    use_these_keys = ['usuarios.tablero11siteID'<br>
                     , 'usuarios.tablero11nomCentroDigital'<br>
                     , 'usuarios.tablero11localidad'<br>
                     , 'usuarios.tablero11nombreDepartamento'<br>
                     , 'usuarios.tablero11nombreMunicipio'<br>
                     , 'usuarios.tablero11idBeneficiario'<br>
                     , 'usuarios.tablero11location'<br>
                     , 'usuarios.tablero11fechaControl'<br>
                     , 'usuarios.tablero11fecha'<br>
                     , 'usuarios.tablero11anyo'<br>
                     , 'usuarios.tablero11mes'<br>
                     , '@timestamp']

    conteoDispositivos['@timestamp'] = now.isoformat()

In [ ]:
    
#     #Cambio de variables utilizando los campos del Tablero...
    
#     conteoDispositivos=cambioVariable(conteoDispositivos)
    
#     #2307
    
#     def doc_generator(df):
#         df_iter = df.iterrows()
#         for index, document in df_iter:
#             yield {
#                     "_index": indice, 
#                     "_id": f"{ 'PromedioDispositivos-' + str(document['usuarios.tablero11siteID']) + '-' + str(document['usuarios.fechaControl'])}",
#                     "_source": filterKeys(document),
#                 }
#     #Se comenta escritura en el indice porque no se necesita esa estructura        
#     #salida = helpers.bulk(es, doc_generator(conteoDispositivos))
#     #print("Fecha: ", now,"- Promedios dispositivos Usuarios:",salida[0])
# except:
#     print("Fecha: ", now,"- Ningun Promedios dispositivos Usuarios para insertar en indice principal")
    

# Marcando si usuario es nuevo o recurrente

* Se toma del dataframe datos_det_conex y se cuenta las veces que cada mac_usuario aparece<br>
* Luego se suma con el historico de conexiones por usuario (indice intermedio ohmyfi-total-conexiones-historico)<br>
* marca como Nuevo(1 conexiÃ³n). <br>
<br>
De este proceso se obtiene el campo:<br>
* usuarios.usuariosNuevos<br>
* usuarios.totales.usuariosNuevos

In[236]:

try:<br>
    datos_recurrencia = datos_det_conex[["fecha","tipodoc","documento"]].groupby(["tipodoc","documento"]).agg(['count']).reset_index()<br>
    datos_recurrencia.columns = datos_recurrencia.columns.droplevel(1)<br>
    datos_recurrencia= datos_recurrencia.rename(columns={'fecha' : 'usuarios_recurrencia_aux'})<br>
    datos_recurrencia['tipodoc'] = datos_recurrencia['tipodoc'].replace('','No especificado')<br>
    datos_recurrencia['documento'] = datos_recurrencia['documento'].replace('','No especificado')<br>
except:<br>
    pass

### Se lee historico

Se lee ohmyfi-total-conexiones-historico, el cual es un indice creado a partir de los datos crudos. Este guarda la cantidad de conexiones realizadas por ese usuario. Para que la consulta funcione correctamente se debe colocar el filtro que usuarios_recurrencia exista.

In[237]:

try:<br>
    total_docs = 30000000<br>
    response = es.search(<br>
        index= parametros.ohmyfi_total_c_index,<br>
        body={<br>
              "_source": ['tipodoc', 'documento', 'usuarios_recurrencia'],<br>
              "query": {<br>
                "bool": {<br>
                  "filter": [<br>
                  {<br>
                    "exists": {<br>
                      "field":"usuarios_recurrencia"<br>
                    }<br>
                  }<br>
                  ]<br>
                }<br>
              }<br>
        },<br>
        size=total_docs<br>
    )<br>
    elastic_docs = response["hits"]["hits"]

In [ ]:
    
#     ya_en_indice = pd.DataFrame([x["_source"] for x in elastic_docs])
    
    
# except:
#     ya_en_indice = pd.DataFrame(columns=['tipodoc', 'documento', 'usuarios_recurrencia'])

In[238]:

if ya_en_indice is None or ya_en_indice.empty:<br>
    datos_recurrencia= datos_recurrencia.rename(columns={'usuarios_recurrencia_aux' : 'usuarios_recurrencia'})<br>
else:<br>
    datos_recurrencia = pd.merge(datos_recurrencia, ya_en_indice, on=['tipodoc', 'documento'],how='left')<br>
    datos_recurrencia.fillna({'usuarios_recurrencia':0},inplace=True)<br>
    datos_recurrencia['usuarios_recurrencia'] = datos_recurrencia['usuarios_recurrencia'] + datos_recurrencia['usuarios_recurrencia_aux']<br>
    datos_recurrencia['usuarios_recurrencia'] = [int(x) for x in datos_recurrencia['usuarios_recurrencia']]

In[239]:

try:<br>
    aux_recurrencia = pd.merge(datos_recurrencia,datos_det_conex, on=['tipodoc','documento'],how='inner' )<br>
    aux_recurrencia = aux_recurrencia[(aux_recurrencia['usuarios_recurrencia']==1)]<br>
    aux_recurrencia.loc[aux_recurrencia['usuarios_recurrencia']==1,'usuarios.usuariosNuevos']='Nuevo'<br>
    #aux_recurrencia.loc[aux_recurrencia['usuarios_recurrencia']!=1,'usuarios.usuariosNuevos']='Recurrente'<br>
    aux_recurrencia = aux_recurrencia.rename(columns={'usuarios_recurrencia' :'usuarios.sesiones_Usuarios'})<br>
    aux_recurrencia = aux_recurrencia[["fecha_control","site_id","usuarios.usuariosNuevos",'documento']].groupby(["fecha_control","site_id","usuarios.usuariosNuevos"])['documento'].nunique().reset_index()<br>
    aux_recurrencia= aux_recurrencia.rename(columns={'documento' : 'usuarios.totales.usuariosNuevos'})<br>
except:<br>
    pass

In[240]:

try:<br>
    datos_recurrencia = pd.merge(datos_semilla,  aux_recurrencia, on='site_id', how='inner')<br>
    datos_recurrencia = datos_recurrencia.rename(columns={'fecha_control' : 'usuarios.fechaControl'})<br>
    datos_recurrencia["usuarios.fecha"] = datos_recurrencia["usuarios.fechaControl"].str.split(" ", n = 1, expand = True)[0]<br>
    datos_recurrencia["usuarios.anyo"] = datos_recurrencia["usuarios.fecha"].str[0:4]<br>
    datos_recurrencia["usuarios.mes"] = datos_recurrencia["usuarios.fecha"].str[5:7]<br>
    datos_recurrencia["usuarios.dia"] = datos_recurrencia["usuarios.fecha"].str[8:10]<br>
    datos_recurrencia["usuarios.hora"] = datos_recurrencia["usuarios.fechaControl"].str.split(" ", n = 1, expand = True)[1].str.split(":", n = 2, expand = True)[0]<br>
    datos_recurrencia["usuarios.minuto"] = datos_recurrencia["usuarios.fechaControl"].str.split(" ", n = 1, expand = True)[1].str.split(":", n = 2, expand = True)[1]<br>
    datos_recurrencia= datos_recurrencia.rename(columns={'site_id' : 'usuarios.siteID'})

In [ ]:
    
#     datos_recurrencia['nombreDepartamento'] = datos_recurrencia['usuarios.nombreDepartamento']
#     datos_recurrencia['nombreMunicipio'] = datos_recurrencia['usuarios.nombreMunicipio']
#     datos_recurrencia['idBeneficiario'] = datos_recurrencia['usuarios.idBeneficiario']
#     datos_recurrencia['fecha'] = datos_recurrencia['usuarios.fecha']
#     datos_recurrencia['anyo'] = datos_recurrencia['usuarios.anyo']
#     datos_recurrencia['mes'] = datos_recurrencia['usuarios.mes']
#     datos_recurrencia['dia'] = datos_recurrencia['usuarios.dia']
# except:
#     pass

# Insertando recurrencia de usuario en indice principal

la lista use_these_keys se usa para referenciar cuales campos del dataframe irÃ¡n al indice final. si los datos no se declaran en este, no se insertarÃ¡n

In[241]:

try:<br>
    use_these_keys = ['usuarios.tablero11.siteID'<br>
                      , 'usuarios.tablero11.nomCentroDigital'<br>
                      , 'usuarios.tablero11.localidad'<br>
                      , 'usuarios.tablero11.nombreDepartamento'<br>
                      , 'usuarios.tablero11.nombreMunicipio'<br>
                      , 'usuarios.tablero11.idBeneficiario'<br>
                      , 'usuarios.tablero11.location'<br>
    #                   , 'usuarios.tablero11sesiones_Usuarios'<br>
    #                   , 'usuarios.tablero11consumoUsuarios'<br>
    #                   , 'usuarios.tablero11apGroup'<br>
                      , 'usuarios.tablero11.fechaControl'<br>
                      , 'usuarios.tablero11.fecha'<br>
                      , 'usuarios.tablero11.anyo'<br>
                      , 'usuarios.tablero11.mes'<br>
                      , '@timestamp']

    datos_recurrencia['@timestamp'] = now.isoformat()

In [ ]:
    
#     datos_recurrencia=cambioVariable(datos_recurrencia)
    
#     datos_recurrencia=cambio
    
#     def doc_generator(df,use_these_keys):
#         df_iter = df.iterrows()
#         for index, document in df_iter:
#             yield {
#                     "_index": indice, 
#                     "_id": f"{ 'Recurrencia-' + str(document['usuarios.tablero11.siteID']) + '-' + str(document['usuarios.tablero11.fechaControl'])}",
#                     "_source": filterKeys(document,use_these_keys),
#                 }
            
#     #Se comenta escritura en el indice porque no se necesita esa estructura        
#     #  salida = helpers.bulk(es, doc_generator(datos_recurrencia))
# #     print("Fecha: ", now,"- recurrencia de usuario a indice:",salida[0])
# except:
#     print("Fecha: ", now,"- Ninguna recurrencia de usuario para insertar en indice principal")

## Insertando detalle MACs

In[242]:

O Se utilizan campos MAC en el Tablero 11..

use_these_keys = ['usuarios.tablero11siteID'<br>
                  ,'usuarios.tablero11nomCentroDigital'<br>
                  , 'usuarios.tablero11localidad'<br>
                  , 'usuarios.tablero11nombreDepartamento'<br>
                  , 'usuarios.tablero11nombreMunicipio'<br>
                  , 'usuarios.tablero11idBeneficiario'<br>
                  , 'usuarios.tablero11location'<br>
                  , 'usuarios.tablero11apGroup'<br>
                  , 'usuarios.tablero11fechaControl'<br>
                  , 'usuarios.tablero11fecha'<br>
                  , 'usuarios.tablero11anyo'<br>
                  , 'usuarios.tablero11mes'<br>
                  , '@timestamp']

# use_these_keys = ['usuarios.siteID'<br>
#                   , 'usuarios.nomCentroDigital'<br>
#                   , 'usuarios.codISO'<br>
#                   , 'usuarios.localidad'<br>
#                   , 'usuarios.nombreDepartamento'<br>
#                   , 'usuarios.sistemaEnergia'<br>
#                   , 'usuarios.nombreMunicipio'<br>
#                   , 'usuarios.idBeneficiario'<br>
#                   , 'usuarios.location'<br>
#                   , 'usuarios.macRed'<br>
#                   , 'usuarios.apGroup'<br>
#                   , 'usuarios.dispositivo.mac'<br>
#                   , 'usuarios.dispositivo.tipo'<br>
#                   , 'usuarios.dispositivo.nombre'<br>
#                   , 'usuarios.dispositivo.marca'<br>
#                   , 'usuarios.dispositivo.tecnologia'<br>
#                   , 'usuarios.dispositivo.sisOperativo'<br>
#                   , 'usuarios.fechaControl'<br>
#                   , 'usuarios.fecha'<br>
#                   , 'usuarios.anyo'<br>
#                   , 'usuarios.mes'<br>
#                   , 'usuarios.dia'<br>
#                   , 'usuarios.hora'<br>
#                   , 'usuarios.minuto'<br>
#                   , '@timestamp']<br>
def doc_generator_mac(df,use_these_keys):<br>
    df_iter = df.iterrows()<br>
    for index, document in df_iter:<br>
        yield {<br>
                "_index": indice, <br>
                "_id": f"{ 'detalleMAC-' + str(document['usuarios.tablero11siteID']) + '-' + str(document['usuarios.fechaControl']) + '-' + str(document['usuarios.dispositivo.mac'])   }",<br>
                "_source": filterKeys(document,use_these_keys),<br>
            }

## Insertando campos para tablero11

In[243]:

use_these_keys = ['usuarios.tablero11.siteID'<br>
                  ,'usuarios.tablero11.nomCentroDigital'<br>
                  , 'usuarios.tablero11.localidad'<br>
                  , 'usuarios.tablero11.nombreDepartamento'<br>
                  , 'usuarios.tablero11.nombreMunicipio'<br>
                  , 'usuarios.tablero11.idBeneficiario'<br>
                  , 'usuarios.tablero11.location'<br>
                  , 'usuarios.tablero11.sesiones_Usuarios'<br>
                  , 'usuarios.tablero11.consumoUsuarios'<br>
                  , 'usuarios.tablero11.apGroup'<br>
                  , 'usuarios.tablero11.fechaControl'<br>
                  , 'usuarios.tablero11.fecha'<br>
                  , 'usuarios.tablero11.anyo'<br>
                  , 'usuarios.tablero11.mes'<br>
                  , '@timestamp']<br>
def doc_tablero11(df,use_these_keys):<br>
    df_iter = df.iterrows()<br>
    for index, document in df_iter:<br>
        yield {<br>
                "_index": indice, <br>
                "_id": f"{ 'Tablero11-' + str(document['usuarios.tablero11.idBeneficiario']) + '-' + str(document['usuarios.tablero11.fechaControl'])  }",<br>
                "_source": filterKeys(document,use_these_keys),<br>
            }

In[244]:

try:<br>
    dispositivoUsuarios = datos_det_conex[['site_id','usuarios.macRed','usuarios.apGroup'<br>
                                ,'mac_usuario','fecha_control'<br>
                                ,'usuarios.tipoDispositivoUsuarios'<br>
                                ,'name','manufacturer','radio.band'<br>
                                ,'usuarios.sistemaOperativoUsuarios']].drop_duplicates(subset=['mac_usuario'])<br>
    dispositivoUsuarios.fillna({'usuarios.tipoDispositivoUsuarios':'No identificado'<br>
                               ,'name':'No identificado'<br>
                               ,'manufacturer':'No identificado'<br>
                               ,'radio.band':'No identificado'<br>
                               ,'usuarios.sistemaOperativoUsuarios':'No identificado'<br>
                               ,'usuarios.macRed':'No identificado'<br>
                               ,'usuarios.apGroup':'No identificado'<br>
                               },inplace=True)

    dispositivoUsuarios = pd.merge(dispositivoUsuarios, datos_semilla, on='site_id',how='inner')<br>
    dispositivoUsuarios= dispositivoUsuarios.rename(columns={<br>
                                                     'site_id' : 'usuarios.siteID'<br>
                                                    ,'mac_usuario' : 'usuarios.dispositivo.mac'<br>
                                                    ,'usuarios.tipoDispositivoUsuarios' : 'usuarios.dispositivo.tipo'<br>
                                                    ,'name' : 'usuarios.dispositivo.nombre'<br>
                                                    ,'manufacturer' : 'usuarios.dispositivo.marca'<br>
                                                    ,'radio.band' : 'usuarios.dispositivo.tecnologia'<br>
                                                    ,'usuarios.sistemaOperativoUsuarios' : 'usuarios.dispositivo.sisOperativo'<br>
                                                    ,'fecha_control':'usuarios.fechaControl'<br>
                                                    })<br>
    dispositivoUsuarios["usuarios.fecha"] = dispositivoUsuarios["usuarios.fechaControl"].str.split(" ", n = 1, expand = True)[0]<br>
    dispositivoUsuarios["usuarios.anyo"] = dispositivoUsuarios["usuarios.fecha"].str[0:4]<br>
    dispositivoUsuarios["usuarios.mes"] = dispositivoUsuarios["usuarios.fecha"].str[5:7]<br>
    dispositivoUsuarios["usuarios.dia"] = dispositivoUsuarios["usuarios.fecha"].str[8:10]<br>
    dispositivoUsuarios["usuarios.hora"] = dispositivoUsuarios["usuarios.fechaControl"].str.split(" ", n = 1, expand = True)[1].str.split(":", n = 2, expand = True)[0]<br>
    dispositivoUsuarios["usuarios.minuto"] = dispositivoUsuarios["usuarios.fechaControl"].str.split(" ", n = 1, expand = True)[1].str.split(":", n = 2, expand = True)[1]<br>
    dispositivoUsuarios['@timestamp'] = now.isoformat()

In [ ]:
    
#    ############# aqui hacer cambio de variable  ################
#     ## El dataframe dispositivos usuarios debe contener los campos de use_these_keys de arriba
    
#     dispositivoUsuarios=cambioVariable(dispositivoUsuarios)
    
    
#     salida = helpers.bulk(es, doc_tablero11(dataframeresultante))
#     print("Fecha: ", now,"- Detalle MACs a indice:",salida[0])

except:<br>
    print("Fecha: ", now,"- NingÃºn Detalle MACs para insertar en indice principal")

### Guardando fecha para control de ejecuciÃ³n

* Se actualiza la fecha de control. Si el calculo supera la fecha hora actual, se asocia esta ultima.

In[245]:

In [ ]:
fecha_ejecucion = (datetime.strptime(fecha_max_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(minutes=50)).strftime("%Y-%m-%d %H:%M:%S")[0:15] + '0:00'    

In [ ]:
if fecha_ejecucion > str(now.strftime('%Y-%m-%d %H:%M:%S'))[0:15] + '0:00':
    fecha_ejecucion = str(now.strftime('%Y-%m-%d %H:%M:%S'))[0:15] + '0:00'
response = es.index(
        index = indice_control,
        id = 'jerarquia-tablero09',
        body = { 'jerarquia-tablero09': 'jerarquia-tablero09','tablero09.fechaControl' : fecha_ejecucion}
)
print("actualizada fecha control de ejecucion:",fecha_ejecucion)

In[ ]: